In [1]:
import qcodes as qc
import numpy as np
from time import sleep
import sweep 
from barreralabdrivers.utils.param_utils import paramp
from cappy.config import BGBSO1_1, COOL_STATION_CONFIG
from cappy.balancer import Balancer

from qcodes.validators import Numbers, Callable, Validator
from qcodes.parameters import Parameter, ScaledParameter
from cappy.virtual_params import Diff, Sum

#Finding instruments which are connected
from pyvisa import ResourceManager
rm = ResourceManager()
print(rm.list_resources())


('TCPIP0::169.254.169.3::inst0::INSTR', 'ASRL1::INSTR', 'ASRL3::INSTR', 'ASRL4::INSTR', 'ASRL6::INSTR', 'GPIB0::15::INSTR')


In [3]:
qc.Instrument.close_all()

container = qc.Station(config_file=str(COOL_STATION_CONFIG))

acdac = container.load_instrument("acdac")
li1 = container.load_instrument("lockin1")
li2 = container.load_instrument("lockin2")


Connected to: BARRERA ACDAC02 (serial:AD9106, firmware:2.00) in 3.04s
Connected to: Stanford_Research_Systems SR865A (serial:005296, firmware:V1.51) in 0.13s
Connected to: Stanford_Research_Systems SR860 (serial:005276, firmware:V1.51) in 0.13s


In [18]:
vdriveamp = li1.amplitude
vdrivephase = acdac.ch1.phase
vdrivephase(0)
vdriveamp(0)


vrefamp = li2.amplitude
vrefphase = acdac.ch2.phase
vrefphase(0)
vdriveamp(0)
vref = (vrefamp, vrefphase)
vdrive = (vdriveamp, vdrivephase)
acdac.ch1.voltage(400)
acdac.ch2.voltage(400)

freq = acdac.frequency
lireference = None
integration_time = 4
balance = Balancer(vref, lireference, vdrive, li1, freq, integration_time)

DRIVE = 10e-3
REF = 10e-3
FREQ = 10777
balance.excite(DRIVE, REF, FREQ)




2 channel mode in use


In [65]:
INIT = (10e-3, 10e-3)
DELTA = (20e-3, 20e-3)
NULL = True
CSTAND = 100e-9 #F
balance.balanceconfig(INIT, DELTA, NULL, CSTAND)
balance.balance()
cap1 = balance.calculate_capacitance()

from cappy.balancer import Capacitance

sleep(1)
cap= Capacitance(balance)

offbal = True
rebal = True
cap.capconfig(offbal, rebal)
cap2 = cap()

print(f'The difference between balancing and offbal is {cap1 - cap2}')  
print(f'cap 1 = {cap1}, and cap 2 = {cap2}')



Kr1, Kr2 = (np.float64(-0.0024373546466400003), np.float64(0.48935057130000004))
Kc1, Kc2 = (np.float64(-0.49803268884755), np.float64(-0.0012930016999999905))
P = 1.2931063873648675e-05
The difference between balancing and offbal is 7.683127262614718e-11
cap 1 = 1.0133485231222305e-07, and cap 2 = 1.012580210395969e-07


In [17]:
acdac.ch2.phase(179.79)
li1.amplitude()
sleep(0.5)
li2.amplitude()

0.00099999993108

In [68]:

cap()


np.float64(1.003501107642538e-07)